In [2]:
import os
import ast
import time
import json
import spacy
import openai
import random
import jellyfish
import numpy as np
import pandas as pd
from umap import UMAP
import seaborn as sns
from nltk.tokenize import word_tokenize 
from datetime import datetime,date,timedelta
from bertopic import BERTopic
from sklearn.metrics import silhouette_score
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
# nltk.download('stopwords')
import matplotlib.pyplot as plt
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from sklearn.linear_model import LinearRegression

In [7]:
df = pd.read_excel('protein_category_detailed_20230322v1.xlsx')

In [11]:
df['Product'] = df['Review ID'].str.split('R',expand=True)[0]

In [5]:
df[df['L2 Cluster']=='Flavor']['L3 Cluster'].unique()

array(['Miscellaneous', 'lime', 'berry', 'mango', 'cola', 'blueberry',
       'fruit', 'orange', 'litchi', 'pineapple', 'coconut', 'guava',
       'chocolate', 'vanilla', 'ghana', 'lemon'], dtype=object)

In [47]:
# f = ['mango','chocolate','blueberry','vanilla','cola','coconut','fruit','orange','litchi','pineapple','ghana',
#      'berry','guava','lemon','lime']
f=['chocolate','mango','vanilla','royale','strawberry','butter','cocoa','alphonso','coffee','coco','pista']

In [48]:
for i in df[df['L2 Cluster']=='Flavor'].index:
#     print(i)
    for word in df['L4 Phrase'][i].split():
        if word.lower() in f:
            df['L3 Cluster'][i] = word.lower()
            break
        else:
            df['L3 Cluster'][i] = 'Miscellaneous'

In [12]:
x = df[df['L2 Cluster']=='Flavor'].loc[:,['L3 Cluster','Review ID','Product']].drop_duplicates().groupby(['L3 Cluster','Product']).count().sort_values(['Review ID'],ascending=False).reset_index()

In [13]:
y = df[(df['L2 Cluster']=='Flavor')&(df['Sentiment']=='Positive')].loc[:,['L3 Cluster','Review ID','Product']].drop_duplicates().groupby(['L3 Cluster','Product']).count().sort_values(['Review ID'],ascending=False).reset_index()

In [16]:
z = pd.merge(x,y,on=['L3 Cluster','Product'])
z.columns = ['Flavor','Product','#Reviews','Positive Reviews']
z.sort_values(['Flavor','Product','#Reviews']).to_excel('protien_product_flavor_summary_20230324v1.xlsx',index=False)

In [167]:
df['L2 Cluster'].replace({"Customerservice":"Customer Service","Smell":"Odor"},inplace=True)

In [173]:
dP1 = d_Protein = {'Performance':['Outcome','Usage','Activity','Customer Service','Quality','Competitor'],
      'Product Attributes':['Taste','Flavor','Ingredient','Design','Odor'],
            'Customer':['Value','Trust','Demographic','Brand','Motivation']}

for l2 in df['L2 Cluster'].unique():
#     print(l2)
    dfx = df[df['L2 Cluster']==l2]
    l1 = ''
    for key,values in dP1.items():
        if l2 in values:
            l1 = key
            break
    for i in dfx.index:
        df['L1 Cluster'][i] = l1

Product Attributes Positive
Product Attributes Negative
Product Attributes Neutral
Performance Positive
Performance Negative
Performance Neutral
Customer Positive
Customer Negative
Customer Neutral


In [174]:
for l2 in ['Odor']:
    for senti in ['Positive','Negative','Neutral']:
        print(l2,senti)
        dfx = df[(df['L2 Cluster']==l2)&(df['Sentiment']==senti)]
        pc = len(dfx)
        rc = dfx['Review ID'].nunique()
        apr = dfx['Review Rating'].mean()
        arr = dfx.drop_duplicates(['Review ID'])['Review Rating'].mean()
        for i in dfx.index:
            df['L2 Phrase Count'][i] = pc
            df['L2 Review Count'][i] = rc
            df['L2 Rating (Phrase)'][i] = apr
            df['L2 Rating (Review)'][i] = arr

Odor Positive
Odor Negative
Odor Neutral


In [3]:
df = pd.read_excel('foundation_category_detailed_20230316v1.xlsx')

In [4]:
d_Protein = { 'Customer Demographics':['Value','Skin Type','Repurchase','Motivation','Emotion','Demographic','Brand'],
         'Attributes':['Time','Partner','Odor','Ingredients','Product Design','Color','Application'],
        'Product Performance':['Finish','Coverage','Competitor']}

for l2 in df['L2 Cluster'].unique():
#     print(l2)
    dfx = df[df['L2 Cluster']==l2]
    l1 = ''
    for key,values in dP1.items():
        if l2 in values:
            l1 = key
            break
    for i in dfx.index:
        df['L1 Cluster'][i] = l1

In [175]:
def id_creation(sheet5,layer,client):
    if layer==1:
        id = 1
        for u_cluster in sheet5['L1 Cluster'].unique():
            for i in sheet5[sheet5['L1 Cluster']==u_cluster].index:
                sheet5['L1 Cluster ID'][i] = client+'L1I'+ str(id)
            id+=1
    elif layer==2 or layer==3:
        a = 'L'+str(layer-1)+' Cluster ID'
        b = 'L'+str(layer)+' Cluster'
        id = 1
        for ul1_id in sheet5[a].unique():
            for ul2_cluster in sheet5[sheet5[a]==ul1_id][b].unique():
                for i in sheet5[(sheet5[a]==ul1_id) & (sheet5[b]==ul2_cluster)].index:
                    sheet5[b+' ID'][i] = client+'L'+str(layer)+'I'+ str(id)
                id+=1
    else:
        id = 1
        for i in sheet5.index:
            sheet5['L4 ID'][i] = client+'L4I'+ str(id)
            id+=1
    return sheet5

In [176]:
for la in range(1,4):
    df = id_creation(df,layer=la,client='protein')

# Detailed Output Without Sentiment

In [12]:
def most_similar_phrase(df1):
    df1['L3 Cluster Phrase']=''
    df1['sim'] = ''
#     df1.drop(['index'],1,inplace=True)
#Calculating the mean similarity score corresponding to each phrase & then taking the phrase with max mean similarity score
    dfx = df1
    if len(dfx)==1:
        for j in dfx.index:
            df1['L3 Cluster Phrase'][j] = dfx['L4 Phrase'].iloc[0].strip().capitalize()                   
        return df1

    for i in dfx.index:
        sim=[]
        for j in dfx.index:
            if i!=j:
                sim.append(jellyfish.levenshtein_distance(df1['L4 Phrase'][i],df1['L4 Phrase'][j]))
        df1['sim'][i] = sum(sim)/len(sim)

    m = df1['sim'].min()
    p = list(df1[(df1['sim']==m)]['L4 Phrase'])[0]
    for i in dfx.index:
        df1['L3 Cluster Phrase'][i] = p.strip().capitalize()                   
#     print(df1.loc[:,['Cluster','Sentiment','L3 Cluster Phrase']].sort_values(['Cluster','Sentiment'],ascending=False).drop_duplicates(),'\n')
    return df1

In [2]:
df = pd.read_excel('hair_oil_detailed_20230223v4.xlsx')

In [3]:
#Category File
for l in ['L1','L2','L3']:
    for id in df[l+' Cluster ID'].unique():
        print(l,id)
        dfx = df[(df[l+' Cluster ID']==id)]
        pc = len(dfx)
        rc = dfx['Review ID'].nunique()
        apr = dfx['Review Rating'].mean()
        arr = dfx.drop_duplicates(['Review ID'])['Review Rating'].mean()
        for i in dfx.index:
            df[l+' Phrase Count'][i] = pc
            df[l+' Review Count'][i] = rc
            df[l+' Rating (Phrase)'][i] = apr
            df[l+' Rating (Review)'][i] = arr

L1 hair_oilL1I2
L1 hair_oilL1I1
L1 hair_oilL1I4
L1 hair_oilL1I3
L2 hair_oilL2I4
L2 hair_oilL2I5
L2 hair_oilL2I8
L2 hair_oilL2I7
L2 hair_oilL2I6
L2 hair_oilL2I9
L2 hair_oilL2I1
L2 hair_oilL2I2
L2 hair_oilL2I18
L2 hair_oilL2I16
L2 hair_oilL2I17
L2 hair_oilL2I10
L2 hair_oilL2I14
L2 hair_oilL2I12
L2 hair_oilL2I13
L2 hair_oilL2I15
L2 hair_oilL2I11
L3 hair_oilL3I35
L3 hair_oilL3I25
L3 hair_oilL3I22
L3 hair_oilL3I32
L3 hair_oilL3I38
L3 hair_oilL3I26
L3 hair_oilL3I24
L3 hair_oilL3I30
L3 hair_oilL3I34
L3 hair_oilL3I40
L3 hair_oilL3I36
L3 hair_oilL3I27
L3 hair_oilL3I31
L3 hair_oilL3I28
L3 hair_oilL3I33
L3 hair_oilL3I29
L3 hair_oilL3I37
L3 hair_oilL3I23
L3 hair_oilL3I45
L3 hair_oilL3I77
L3 hair_oilL3I76
L3 hair_oilL3I65
L3 hair_oilL3I73
L3 hair_oilL3I68
L3 hair_oilL3I67
L3 hair_oilL3I74
L3 hair_oilL3I71
L3 hair_oilL3I239
L3 hair_oilL3I153
L3 hair_oilL3I158
L3 hair_oilL3I149
L3 hair_oilL3I79
L3 hair_oilL3I103
L3 hair_oilL3I101
L3 hair_oilL3I91
L3 hair_oilL3I89
L3 hair_oilL3I102
L3 hair_oilL3I104
L

In [22]:
for l3 in df['L3 Cluster ID'].unique():
    print(l3)
    dfx = df[df['L3 Cluster ID']==l3]
    dfx = most_similar_phrase(dfx)
    ph = dfx['L3 Cluster Phrase'].iloc[0]
    for i in df[df['L3 Cluster ID']==l3].index:
        df['L3 Cluster Phrase'][i] = ph

slim_shakeL3I1
slim_shakeL3I2
slim_shakeL3I3
slim_shakeL3I4
slim_shakeL3I5
slim_shakeL3I6
slim_shakeL3I7
slim_shakeL3I8
slim_shakeL3I9
slim_shakeL3I10
slim_shakeL3I11
slim_shakeL3I12
slim_shakeL3I13
slim_shakeL3I14
slim_shakeL3I15
slim_shakeL3I16
slim_shakeL3I17
slim_shakeL3I18
slim_shakeL3I19
slim_shakeL3I20
slim_shakeL3I21
slim_shakeL3I22
slim_shakeL3I23
slim_shakeL3I24
slim_shakeL3I25
slim_shakeL3I26
slim_shakeL3I27
slim_shakeL3I28
slim_shakeL3I29
slim_shakeL3I30
slim_shakeL3I31
slim_shakeL3I32
slim_shakeL3I33
slim_shakeL3I34
slim_shakeL3I35
slim_shakeL3I36
slim_shakeL3I37
slim_shakeL3I38
slim_shakeL3I39
slim_shakeL3I40
slim_shakeL3I41
slim_shakeL3I42
slim_shakeL3I43
slim_shakeL3I44
slim_shakeL3I45
slim_shakeL3I46
slim_shakeL3I47
slim_shakeL3I48
slim_shakeL3I49
slim_shakeL3I50
slim_shakeL3I51
slim_shakeL3I52
slim_shakeL3I53
slim_shakeL3I54
slim_shakeL3I55
slim_shakeL3I56
slim_shakeL3I57
slim_shakeL3I58
slim_shakeL3I59
slim_shakeL3I60
slim_shakeL3I61
slim_shakeL3I62
slim_shakeL3I63
s

In [53]:
#Product Level
df['Product'] = df['Review ID'].str.split('R',expand=True)[0]

for p in df['Product'].unique():
    for l in ['L1','L2','L3']:
        for id in df[df['Product']==p][l+' Cluster ID'].unique():
            print(p,l,id)
            dfx = df[(df['Product']==p)&(df[l+' Cluster ID']==id)]
            pc = len(dfx)
            rc = dfx['Review ID'].nunique()
            apr = dfx['Review Rating'].mean()
            arr = dfx.drop_duplicates(['Review ID'])['Review Rating'].mean()
            for i in dfx.index:
                df[l+' Phrase Count'][i] = pc
                df[l+' Review Count'][i] = rc
                df[l+' Rating (Phrase)'][i] = apr
                df[l+' Rating (Review)'][i] = arr

PAIN1 L1 slim_shakeL1I1
PAIN1 L1 slim_shakeL1I2
PAIN1 L1 slim_shakeL1I3
PAIN1 L2 slim_shakeL2I1
PAIN1 L2 slim_shakeL2I2
PAIN1 L2 slim_shakeL2I8
PAIN1 L2 slim_shakeL2I13
PAIN1 L2 slim_shakeL2I9
PAIN1 L2 slim_shakeL2I3
PAIN1 L2 slim_shakeL2I14
PAIN1 L2 slim_shakeL2I10
PAIN1 L2 slim_shakeL2I16
PAIN1 L2 slim_shakeL2I5
PAIN1 L2 slim_shakeL2I11
PAIN1 L2 slim_shakeL2I18
PAIN1 L2 slim_shakeL2I6
PAIN1 L2 slim_shakeL2I7
PAIN1 L2 slim_shakeL2I12
PAIN1 L3 slim_shakeL3I1
PAIN1 L3 slim_shakeL3I2
PAIN1 L3 slim_shakeL3I3
PAIN1 L3 slim_shakeL3I5
PAIN1 L3 slim_shakeL3I6
PAIN1 L3 slim_shakeL3I7
PAIN1 L3 slim_shakeL3I8
PAIN1 L3 slim_shakeL3I9
PAIN1 L3 slim_shakeL3I10
PAIN1 L3 slim_shakeL3I11
PAIN1 L3 slim_shakeL3I12
PAIN1 L3 slim_shakeL3I13
PAIN1 L3 slim_shakeL3I14
PAIN1 L3 slim_shakeL3I15
PAIN1 L3 slim_shakeL3I16
PAIN1 L3 slim_shakeL3I17
PAIN1 L3 slim_shakeL3I18
PAIN1 L3 slim_shakeL3I19
PAIN1 L3 slim_shakeL3I20
PAIN1 L3 slim_shakeL3I21
PAIN1 L3 slim_shakeL3I22
PAIN1 L3 slim_shakeL3I23
PAIN1 L3 slim_shake

PAUS1 L1 slim_shakeL1I2
PAUS1 L1 slim_shakeL1I3
PAUS1 L2 slim_shakeL2I1
PAUS1 L2 slim_shakeL2I2
PAUS1 L2 slim_shakeL2I8
PAUS1 L2 slim_shakeL2I13
PAUS1 L2 slim_shakeL2I9
PAUS1 L2 slim_shakeL2I3
PAUS1 L2 slim_shakeL2I14
PAUS1 L2 slim_shakeL2I15
PAUS1 L2 slim_shakeL2I10
PAUS1 L2 slim_shakeL2I4
PAUS1 L2 slim_shakeL2I16
PAUS1 L2 slim_shakeL2I5
PAUS1 L2 slim_shakeL2I11
PAUS1 L2 slim_shakeL2I17
PAUS1 L2 slim_shakeL2I18
PAUS1 L2 slim_shakeL2I6
PAUS1 L2 slim_shakeL2I7
PAUS1 L2 slim_shakeL2I12
PAUS1 L3 slim_shakeL3I1
PAUS1 L3 slim_shakeL3I2
PAUS1 L3 slim_shakeL3I3
PAUS1 L3 slim_shakeL3I5
PAUS1 L3 slim_shakeL2I1_misc
PAUS1 L3 slim_shakeL3I14
PAUS1 L3 slim_shakeL3I15
PAUS1 L3 slim_shakeL3I6
PAUS1 L3 slim_shakeL3I11
PAUS1 L3 slim_shakeL3I10
PAUS1 L3 slim_shakeL3I9
PAUS1 L3 slim_shakeL3I7
PAUS1 L3 slim_shakeL3I13
PAUS1 L3 slim_shakeL3I16
PAUS1 L3 slim_shakeL3I12
PAUS1 L3 slim_shakeL3I17
PAUS1 L3 slim_shakeL3I8
PAUS1 L3 slim_shakeL3I20
PAUS1 L3 slim_shakeL3I19
PAUS1 L3 slim_shakeL3I21
PAUS1 L3 slim_s

# Regression Analysis

In [134]:
daf = pd.read_excel('foundation_completion_20230302v1.xlsx')
daf = daf.drop_duplicates(['Product','Aspect','Sentiment','Review ID']).groupby(['Product','Aspect','Sentiment']).mean()
df = pd.read_excel('foundation_all_reviews_20230228v1.xlsx')
df = df.groupby(['Product']).mean().loc[:,['Rating']]
df = pd.merge(daf.reset_index(),df.reset_index(),on=['Product'])
df.columns = ['Product','Aspect','Sentiment','L2 Rating','Product Rating']

aspects = list(df['Aspect'].unique())
daf = pd.DataFrame(columns = ['Product']+['Sentiment']+aspects+['Product Rating'])
daf['Product'] = df.loc[:,['Product','Sentiment']].drop_duplicates().reset_index(drop=True)['Product']
daf['Sentiment'] = df.loc[:,['Product','Sentiment']].drop_duplicates().reset_index(drop=True)['Sentiment']
# daf['Product'] = pd.Series(df['Product'].unique())

for i in daf.index:
    for aspect in aspects:
        try:
            daf[aspect][i] = df[(df['Product']==daf['Product'][i])&(df['Aspect']==aspect)&(df['Sentiment']==daf['Sentiment'][i])]['L2 Rating'].iloc[0]
        except:
            daf[aspect][i] = 0
        daf['Product Rating'][i] = df[(df['Product']==daf['Product'][i])]['Product Rating'].iloc[0]

X = daf[daf['Sentiment']=='Positive'].loc[:,aspects]
y = daf[daf['Sentiment']=='Positive']['Product Rating']
reg = LinearRegression().fit(X, y)
corr1 = pd.DataFrame(columns = ['Sentiment','Aspect','Coefficient'])
corr1['Aspect'] = X.columns
corr1['Sentiment'] = 'Positive'
corr1['Coefficient'] = reg.coef_

X = daf[daf['Sentiment']=='Negative'].loc[:,aspects]
y = daf[daf['Sentiment']=='Negative']['Product Rating']
reg = LinearRegression().fit(X, y)
corr2 = pd.DataFrame(columns = ['Sentiment','Aspect','Coefficient'])
corr2['Aspect'] = X.columns
corr2['Sentiment'] = 'Negative'
corr2['Coefficient'] = reg.coef_

corr = corr.append(corr2,ignore_index=True).sort_values(['Aspect','Sentiment'])

In [142]:
daf = pd.read_excel('foundation_completion_20230302v1.xlsx')
x = daf.drop_duplicates(['Aspect','Sentiment','Review ID']).groupby(['Aspect','Sentiment']).mean()
daf = daf.drop_duplicates(['Aspect','Sentiment','Review ID']).groupby(['Aspect','Sentiment']).count().loc[:,['Review ID']].join(x).sort_values(['Review ID'],ascending=False)#.to_excel('foundation_l2_analysis_20230321v1.xlsx')

In [ ]:
pd.merge(daf.reset_index(),corr,on=['Aspect','Sentiment']).sort_values(['Aspect','Sentiment'],ascending=False).to_excel('foundation_l2_analysis_20230321v1.xlsx',index=False)

In [ ]:
#Modules List:
# 1. Daily Revies
# 2. Top n% File
# 3. Making Product File from category file
# 4. Combining L3 Clusters
# 5. Competitor Extraction
# 6. Monthly Review Count
# 7. L3 Clusters Cross Connection

In [3]:
df = pd.read_excel('protein_category_detailed_20230318v1.xlsx')

In [7]:
scores=[]
for i in df['L2 Cluster'].unique():
    daf = df[(df['L2 Cluster']==i)]
    vectorizer = TfidfVectorizer(ngram_range = (1,3),stop_words="english")
    dv = vectorizer.fit_transform(daf['L4 Phrase'])
    try:
      scores.append(silhouette_score(dv,daf.loc[:,['L3 Cluster']]))
      print(i,scores[-1])
    except:
        pass
print("Average Score:",sum(scores)/len(scores))

Taste 0.20400971028540715
Outcome 0.04695965561848166
Flavor 0.13883360120045385
Value 0.22132970621884054
Ingredient 0.0333830287754678
Usage 0.0566091543867892
Activity 0.18921094894297608
Trust 0.050095486073447874
Demographic 0.07901463337695139
Design 0.07951805177917799
Customerservice 0.02324964587122203
Brand 0.017081639870337652
Competitor 0.005463287136065998
Average Score: 0.08805834996427842


In [74]:
for p in df['Product'].unique():
    df[df['Product']==p].to_csv(p+'_detailed_L4_20230318v1.csv',index=False)

# Daily Reviews

In [42]:
for p in df['Product'].unique():
    daf = df[df['Product']==p].join(pd.get_dummies(df[df['Product']==p]['Rating'])).groupby('Date').sum().iloc[:,2:]
    daf = daf.join(df[df['Product']==p].groupby('Date').count().loc[:,['ReviewID']])
    daf.rename(columns = {'ReviewID':'reviews'}, inplace = True)
    l = list(daf.columns)[:-1]
    for i in l:
        name = '% of '+str(i)
        daf[name] = round(100*(daf[i]/daf['reviews']),1)
    daf.reset_index(inplace=True)
    daf.columns = ["Date","1stars","2stars","3stars","4stars","5stars","reviews","PercentOne","PercentTwo","PercentThree","PercentFour",
                   "PercentFive"]
    daf.to_csv(p+'_protein_daily_reviews_20230318v1.csv',index=False)

# Top n% File

In [6]:
# df = pd.read_excel('protein_category_detailed_20230318v1.xlsx')
daf = pd.DataFrame(columns = ['L2 Cluster','Total Reviews','Miscellaneous Reviews','Misc. %'])
daf = daf.append(df.loc[:,['L2 Cluster']].drop_duplicates(),ignore_index=True)

In [7]:
for i in daf.index:
    dfx = df[(df['L2 Cluster']==daf['L2 Cluster'][i])]
    daf['Total Reviews'][i] = dfx['Review ID'].nunique()
    daf['Miscellaneous Reviews'][i] = dfx[dfx['L3 Cluster']=='Miscellaneous']['Review ID'].nunique()
    daf['Misc. %'][i] = daf['Miscellaneous Reviews'][i]*100/daf['Total Reviews'][i]
    
for i in ['Top 5','5_%','Top 10','10_%','Top 15','15_%','Top 20','20_%','Top 25','25_%']:
    daf[i] = ''
    
for i in daf.index:
    dfx = df[(df['L2 Cluster']==daf['L2 Cluster'][i])]
    for j in [5,10,15,20,25]:
        l = list(dfx[dfx['L3 Cluster']!='Miscellaneous'].loc[:,['L3 Cluster','L3 Review Count']].drop_duplicates().sort_values(['L3 Review Count'],ascending=False)[:j]['L3 Cluster'])
        daf['Top '+str(j)][i] = dfx[dfx['L3 Cluster'].isin(l)]['Review ID'].nunique()
        daf[str(j)+'_%'][i] = daf['Top '+str(j)][i]*100/daf['Total Reviews'][i]

In [9]:
daf.to_excel('hair_oil_%review_analysis_20230323v1.xlsx',index=False)

In [10]:
df['Display Flag']=0
# df = df[~df['L2 Cluster'].isin(['Oily','Texture','Formula'])]
for a in df['L2 Cluster'].unique():
    for senti in ['Positive','Negative']:
        dfx = df[(df['L2 Cluster']==a)]
        l = list(dfx[dfx['L3 Cluster']!='Miscellaneous'].loc[:,['L3 Cluster','L3 Review Count']].drop_duplicates().sort_values(['L3 Review Count'],ascending=False)[:5]['L3 Cluster'])
        for i in dfx[dfx['L3 Cluster'].isin(l)].index:
            df['Display Flag'][i]=1

# Making Product File from Category File

In [75]:
df['Product'] = df['Review ID'].str.split('R',expand=True)[0]

for p in df['Product'].unique():
    for l in ['L1','L2','L3']:
        for id in df[df['Product']==p][l+' Cluster ID'].unique():
            for senti in df[(df['Product']==p)&(df[l+' Cluster ID']==id)]['Sentiment'].unique():
                print(p,l,id,senti)
                dfx = df[(df['Product']==p)&(df[l+' Cluster ID']==id)&(df['Sentiment']==senti)]
                pc = len(dfx)
                rc = dfx['Review ID'].nunique()
                apr = dfx['Review Rating'].mean()
                arr = dfx.drop_duplicates(['Review ID'])['Review Rating'].mean()
                for i in dfx.index:
                    df[l+' Phrase Count'][i] = pc
                    df[l+' Review Count'][i] = rc
                    df[l+' Rating (Phrase)'][i] = apr
                    df[l+' Rating (Review)'][i] = arr

PDAU10 L1 foundationL1I1 Positive
PDAU10 L1 foundationL1I1 Negative
PDAU10 L1 foundationL1I3 Positive
PDAU10 L1 foundationL1I3 Negative
PDAU10 L1 foundationL1I4 Positive
PDAU10 L1 foundationL1I4 Negative
PDAU10 L1 foundationL1I2 Positive
PDAU10 L1 foundationL1I2 Negative
PDAU10 L2 foundationL2I1 Positive
PDAU10 L2 foundationL2I1 Negative
PDAU10 L2 foundationL2I2 Positive
PDAU10 L2 foundationL2I2 Negative
PDAU10 L2 foundationL2I17 Positive
PDAU10 L2 foundationL2I17 Negative
PDAU10 L2 foundationL2I18 Positive
PDAU10 L2 foundationL2I18 Negative
PDAU10 L2 foundationL2I3 Positive
PDAU10 L2 foundationL2I3 Negative
PDAU10 L2 foundationL2I5 Positive
PDAU10 L2 foundationL2I5 Negative
PDAU10 L2 foundationL2I6 Positive
PDAU10 L2 foundationL2I6 Negative
PDAU10 L2 foundationL2I7 Positive
PDAU10 L2 foundationL2I7 Negative
PDAU10 L2 foundationL2I9 Positive
PDAU10 L2 foundationL2I9 Negative
PDAU10 L2 foundationL2I10 Positive
PDAU10 L2 foundationL2I10 Negative
PDAU10 L2 foundationL2I11 Positive
PDAU10 

PDAU10 L3 foundationL3I193 Negative
PDAU10 L3 foundationL3I194 Positive
PDAU10 L3 foundationL3I195 Positive
PDAU10 L3 foundationL3I195 Negative
PDAU10 L3 foundationL3I196 Positive
PDAU10 L3 foundationL3I197 Positive
PDAU10 L3 foundationL3I197 Negative
PDAU10 L3 foundationL3I198 Positive
PDAU10 L3 foundationL3I198 Negative
PDAU10 L3 foundationL3I199 Positive
PDAU10 L3 foundationL3I200 Positive
PDAU10 L3 foundationL3I200 Negative
PDAU10 L3 foundationL3I201 Positive
PDAU10 L3 foundationL3I202 Positive
PDAU10 L3 foundationL3I203 Positive
PDAU10 L3 foundationL3I204 Positive
PDAU10 L3 foundationL3I205 Positive
PDAU10 L3 foundationL3I205 Negative
PDAU10 L3 foundationL3I206 Positive
PDAU10 L3 foundationL3I207 Positive
PDAU10 L3 foundationL3I208 Positive
PDAU10 L3 foundationL3I209 Positive
PDAU10 L3 foundationL3I210 Positive
PDAU10 L3 foundationL3I211 Positive
PDAU10 L3 foundationL3I211 Negative
PDAU10 L3 foundationL3I212 Positive
PDAU10 L3 foundationL3I213 Positive
PDAU10 L3 foundationL3I214 P

PDAU10 L3 foundationL3I356 Positive
PDAU10 L3 foundationL3I356 Negative
PDAU10 L3 foundationL3I357 Positive
PDAU10 L3 foundationL3I358 Positive
PDAU10 L3 foundationL3I359 Positive
PDAU10 L3 foundationL3I359 Negative
PDAU10 L3 foundationL3I360 Positive
PDAU10 L3 foundationL3I360 Negative
PDAU10 L3 foundationL3I361 Positive
PDAU10 L3 foundationL3I362 Positive
PDAU10 L3 foundationL3I362 Negative
PDAU10 L3 foundationL3I363 Positive
PDAU10 L3 foundationL3I363 Negative
PDAU10 L3 foundationL3I364 Positive
PDAU10 L3 foundationL3I364 Negative
PDAU10 L3 foundationL3I365 Positive
PDAU10 L3 foundationL3I365 Negative
PDAU10 L3 foundationL3I366 Positive
PDAU10 L3 foundationL3I366 Negative
PDAU10 L3 foundationL3I367 Positive
PDAU10 L3 foundationL3I368 Positive
PDAU10 L3 foundationL3I368 Negative
PDAU10 L3 foundationL3I369 Positive
PDAU10 L3 foundationL3I370 Positive
PDAU10 L3 foundationL3I370 Negative
PDAU10 L3 foundationL3I371 Positive
PDAU10 L3 foundationL3I372 Positive
PDAU10 L3 foundationL3I372 N

PDAU10 L3 foundationL3I573 Negative
PDAU10 L3 foundationL3I605 Positive
PDAU10 L3 foundationL3I593 Positive
PDAU10 L3 foundationL3I593 Negative
PDAU10 L3 foundationL2I6_misc_P Positive
PDAU10 L3 foundationL3I537 Negative
PDAU10 L3 foundationL3I612 Negative
PDAU10 L3 foundationL3I619 Negative
PDAU10 L3 foundationL3I609 Negative
PDAU10 L3 foundationL3I607 Negative
PDAU10 L3 foundationL3I604 Negative
PDAU10 L3 foundationL3I586 Negative
PDAU10 L3 foundationL3I602 Negative
PDAU10 L3 foundationL3I567 Negative
PDAU10 L3 foundationL3I571 Negative
PDAU10 L3 foundationL3I574 Negative
PDAU10 L3 foundationL3I595 Negative
PDAU10 L3 foundationL3I556 Negative
PDAU10 L3 foundationL3I566 Negative
PDAU10 L3 foundationL3I587 Negative
PDAU10 L3 foundationL3I572 Negative
PDAU10 L3 foundationL3I579 Negative
PDAU10 L3 foundationL2I6_misc_N Negative
PDAU10 L3 foundationL3I620 Positive
PDAU10 L3 foundationL3I620 Negative
PDAU10 L3 foundationL3I621 Positive
PDAU10 L3 foundationL3I622 Positive
PDAU10 L3 foundati

PDAU10 L3 foundationL3I807 Negative
PDAU10 L3 foundationL3I792 Positive
PDAU10 L3 foundationL3I792 Negative
PDAU10 L3 foundationL3I801 Positive
PDAU10 L3 foundationL3I801 Negative
PDAU10 L3 foundationL2I9_misc_P Positive
PDAU10 L3 foundationL3I788 Negative
PDAU10 L3 foundationL3I723 Negative
PDAU10 L3 foundationL3I800 Negative
PDAU10 L3 foundationL3I784 Negative
PDAU10 L3 foundationL3I781 Negative
PDAU10 L3 foundationL3I798 Negative
PDAU10 L3 foundationL3I796 Negative
PDAU10 L3 foundationL3I676 Negative
PDAU10 L3 foundationL2I9_misc_N Negative
PDAU10 L3 foundationL3I809 Positive
PDAU10 L3 foundationL3I810 Positive
PDAU10 L3 foundationL3I811 Positive
PDAU10 L3 foundationL3I812 Positive
PDAU10 L3 foundationL3I813 Positive
PDAU10 L3 foundationL3I814 Positive
PDAU10 L3 foundationL3I815 Positive
PDAU10 L3 foundationL3I816 Positive
PDAU10 L3 foundationL3I816 Negative
PDAU10 L3 foundationL3I817 Positive
PDAU10 L3 foundationL3I818 Positive
PDAU10 L3 foundationL3I819 Positive
PDAU10 L3 foundati

...

PDUS4 L3 foundationL3I149 Positive
PDUS4 L3 foundationL3I149 Negative
PDUS4 L3 foundationL3I295 Positive
PDUS4 L3 foundationL3I297 Positive
PDUS4 L3 foundationL3I230 Positive
PDUS4 L3 foundationL3I273 Positive
PDUS4 L3 foundationL3I289 Positive
PDUS4 L3 foundationL3I233 Positive
PDUS4 L3 foundationL3I282 Positive
PDUS4 L3 foundationL3I291 Positive
PDUS4 L3 foundationL3I292 Positive
PDUS4 L3 foundationL3I261 Positive
PDUS4 L3 foundationL3I190 Positive
PDUS4 L3 foundationL3I258 Positive
PDUS4 L3 foundationL3I258 Negative
PDUS4 L3 foundationL3I235 Positive
PDUS4 L3 foundationL3I178 Positive
PDUS4 L3 foundationL3I178 Negative
PDUS4 L3 foundationL3I158 Positive
PDUS4 L3 foundationL3I158 Negative
PDUS4 L3 foundationL3I279 Positive
PDUS4 L3 foundationL3I279 Negative
PDUS4 L3 foundationL3I268 Positive
PDUS4 L3 foundationL3I145 Positive
PDUS4 L3 foundationL3I266 Positive
PDUS4 L3 foundationL3I210 Positive
PDUS4 L3 foundationL3I216 Positive
PDUS4 L3 foundationL3I294 Positive
PDUS4 L3 foundationL

PDUS4 L3 foundationL3I421 Positive
PDUS4 L3 foundationL3I405 Positive
PDUS4 L3 foundationL3I405 Negative
PDUS4 L3 foundationL3I446 Positive
PDUS4 L3 foundationL3I447 Positive
PDUS4 L3 foundationL3I453 Positive
PDUS4 L3 foundationL3I441 Positive
PDUS4 L3 foundationL3I429 Positive
PDUS4 L3 foundationL3I399 Positive
PDUS4 L3 foundationL3I394 Positive
PDUS4 L3 foundationL3I456 Positive
PDUS4 L3 foundationL3I412 Positive
PDUS4 L3 foundationL3I416 Positive
PDUS4 L3 foundationL3I443 Positive
PDUS4 L3 foundationL3I483 Positive
PDUS4 L3 foundationL3I435 Positive
PDUS4 L3 foundationL3I435 Negative
PDUS4 L3 foundationL3I440 Positive
PDUS4 L3 foundationL3I432 Positive
PDUS4 L3 foundationL3I445 Positive
PDUS4 L3 foundationL3I445 Negative
PDUS4 L3 foundationL3I401 Positive
PDUS4 L3 foundationL3I398 Positive
PDUS4 L3 foundationL3I396 Positive
PDUS4 L3 foundationL3I420 Positive
PDUS4 L3 foundationL3I419 Positive
PDUS4 L3 foundationL3I404 Positive
PDUS4 L3 foundationL3I466 Positive
PDUS4 L3 foundationL

PDUS4 L3 foundationL3I688 Positive
PDUS4 L3 foundationL3I688 Negative
PDUS4 L3 foundationL3I784 Positive
PDUS4 L3 foundationL3I693 Positive
PDUS4 L3 foundationL3I693 Negative
PDUS4 L3 foundationL3I790 Positive
PDUS4 L3 foundationL3I701 Positive
PDUS4 L3 foundationL3I666 Positive
PDUS4 L3 foundationL3I712 Positive
PDUS4 L3 foundationL3I724 Positive
PDUS4 L3 foundationL3I673 Positive
PDUS4 L3 foundationL3I673 Negative
PDUS4 L3 foundationL3I661 Positive
PDUS4 L3 foundationL3I663 Positive
PDUS4 L3 foundationL3I663 Negative
PDUS4 L3 foundationL3I734 Positive
PDUS4 L3 foundationL3I734 Negative
PDUS4 L3 foundationL3I774 Positive
PDUS4 L3 foundationL3I774 Negative
PDUS4 L3 foundationL3I678 Positive
PDUS4 L3 foundationL3I678 Negative
PDUS4 L3 foundationL3I694 Positive
PDUS4 L3 foundationL3I694 Negative
PDUS4 L3 foundationL3I769 Positive
PDUS4 L3 foundationL3I769 Negative
PDUS4 L3 foundationL3I799 Positive
PDUS4 L3 foundationL3I747 Positive
PDUS4 L3 foundationL3I783 Positive
PDUS4 L3 foundationL

PDUS4 L3 foundationL3I1058 Positive
PDUS4 L3 foundationL3I1050 Negative
PDUS4 L3 foundationL3I1080 Negative
PDUS4 L3 foundationL2I19_misc_N Negative
PDUS4 L3 foundationL3I1237 Positive
PDUS4 L3 foundationL3I1088 Positive
PDUS4 L3 foundationL3I1088 Negative
PDUS4 L3 foundationL3I1130 Positive
PDUS4 L3 foundationL3I1130 Negative
PDUS4 L3 foundationL3I1133 Positive
PDUS4 L3 foundationL3I1133 Negative
PDUS4 L3 foundationL3I1114 Positive
PDUS4 L3 foundationL3I1123 Positive
PDUS4 L3 foundationL3I1110 Positive
PDUS4 L3 foundationL3I1110 Negative
PDUS4 L3 foundationL3I1098 Positive
PDUS4 L3 foundationL3I1098 Negative
PDUS4 L3 foundationL3I1086 Positive
PDUS4 L3 foundationL3I1124 Positive
PDUS4 L3 foundationL3I1124 Negative
PDUS4 L3 foundationL3I1232 Positive
PDUS4 L3 foundationL3I1121 Positive
PDUS4 L3 foundationL3I1190 Positive
PDUS4 L3 foundationL3I1093 Positive
PDUS4 L3 foundationL3I1093 Negative
PDUS4 L3 foundationL3I1172 Positive
PDUS4 L3 foundationL3I1126 Positive
PDUS4 L3 foundationL3I1

PDUS4 L3 foundationL3I1264 Positive
PDUS4 L3 foundationL3I1352 Positive
PDUS4 L3 foundationL3I1308 Positive
PDUS4 L3 foundationL3I1289 Positive
PDUS4 L3 foundationL3I1284 Positive
PDUS4 L3 foundationL3I1328 Positive
PDUS4 L3 foundationL3I1278 Positive
PDUS4 L3 foundationL3I1254 Positive
PDUS4 L3 foundationL3I1309 Positive
PDUS4 L3 foundationL3I1335 Positive
PDUS4 L3 foundationL3I1269 Positive
PDUS4 L3 foundationL3I1257 Positive
PDUS4 L3 foundationL3I1322 Positive
PDUS4 L3 foundationL3I1281 Positive
PDUS4 L3 foundationL3I1327 Positive
PDUS4 L3 foundationL3I1314 Positive
PDUS4 L3 foundationL3I1260 Positive
PDUS4 L3 foundationL3I1311 Positive
PDUS4 L3 foundationL3I1293 Positive
PDUS4 L3 foundationL3I1252 Positive
PDUS4 L3 foundationL3I1351 Positive
PDUS4 L3 foundationL3I1326 Positive
PDUS4 L3 foundationL3I1326 Negative
PDUS4 L3 foundationL3I1256 Positive
PDUS4 L3 foundationL3I1320 Positive
PDUS4 L3 foundationL3I1263 Positive
PDUS4 L3 foundationL3I1304 Positive
PDUS4 L3 foundationL3I1318 P

In [42]:
daf.to_excel('Review_count_analysis_20230316v1.xlsx',index=False)

# Combining L3 Clusters

In [69]:
df4 = pd.read_excel('foundation_category_detailed_20230313v1 1.xlsx','L4')
df3 = pd.read_excel('foundation_category_detailed_20230313v1 1.xlsx','L3')
df3 = df3.dropna()
df3['L3 Cluster'] = df3['Combine'].str.capitalize()
df3 = df3.drop_duplicates(['L3 Cluster ID','Combine'])

df4['Combine']=0
for i in df3.index:
    print(i)
    for j in df4[df4['L3 Cluster ID']==df3['L3 Cluster ID'][i]].index:
        df4['Combine'][j] = 1
        df4['L3 Cluster'][j] = df3['L3 Cluster'][i]
        
df4 = id_creation(df4,3,'foundation')

for id in df4[df4['Combine']==1]['L3 Cluster ID'].unique():
    for senti in df4[df4['L3 Cluster ID']==id]['Sentiment'].unique():
        dfx = df4[(df4['L3 Cluster ID']==id)&(df4['Sentiment']==senti)]
#         print(id,senti)
        pc = len(dfx)
        rc = dfx['Review ID'].nunique()
        apr = dfx['Review Rating'].mean()
        arr = dfx.drop_duplicates(['Review ID'])['Review Rating'].mean()
        for i in dfx.index:
            df['L3 Phrase Count'][i] = pc
            df['L3 Review Count'][i] = rc
            df['L3 Rating (Phrase)'][i] = apr
            df['L3 Rating (Review)'][i] = arr

df4['sim'] = ''
for cluster in df4[(df4['Combine']==1)]['L3 Cluster ID'].unique():
    for senti in df4[df4['L3 Cluster ID']==cluster]['Sentiment'].unique():
#         print(cluster,senti)
        dfx = df4[(df4['L3 Cluster ID']==cluster)&(df4.Sentiment==senti)]
        if dfx['L3 Cluster Phrase'].nunique()==1:
            continue
        if len(dfx)==1:
            for j in dfx.index:
                df4['L3 Cluster Phrase'][j] = dfx['L4 Phrase'].iloc[0].strip().capitalize()                   
            continue

        for i in dfx.index:
            sim=[]
            for j in dfx.index:
                if i!=j:
                    sim.append(jellyfish.levenshtein_distance(df4['L4 Phrase'][i],df4['L4 Phrase'][j]))
            df4['sim'][i] = sum(sim)/len(sim)

        m = df4[(df4['L3 Cluster ID']==cluster)&(df4.Sentiment==senti)]['sim'].min()
        p = list(df4[(df4['L3 Cluster ID']==cluster)&(df4.Sentiment==senti)&(df4['sim']==m)]['L4 Phrase'])[0]
        for i in dfx.index:
            df4['L3 Cluster Phrase'][i] = p.strip().capitalize() 

In [5]:
df = cluster_merge(df,False)

In [30]:
def cluster_merge(l4,multi_product,min_l3_clusters=3,min_l3_phrases=10,percentage=10):
    df = pd.DataFrame(columns = list(l4.columns)+['sim'])
    if multi_product:
        ps = l4['Product'].unique()
    else:
        ps = [1]
    for p in ps:
        if multi_product:
            df1 = l4[l4['Product']==p]
            l2s = df1['L2 Cluster ID'].unique()
        else:
            df1 = l4.copy()
            l2s = df1['L2 Cluster ID'].unique()   
        for l2 in l2s:
#             for senti in df1[df1['L2 Cluster ID']==l2]['Sentiment'].unique():
            df2 = df1[(df1['L2 Cluster ID']==l2)]
            if df2['L3 Cluster ID'].nunique()<min_l3_clusters:
                df = df.append(df2,ignore_index=True)
                continue

            l3_phrases = df2.drop_duplicates(['L3 Cluster ID'])['L3 Phrase Count'].mean()
            l3s = df2['L3 Cluster ID'].unique()
            x = []
            for l3 in l3s:
                cnt = df2[df2['L3 Cluster ID']==l3]['L3 Phrase Count'].iloc[0]
                if cnt<((percentage/100)*l3_phrases) or cnt<min_l3_phrases:
                    x.append(l3)
            if len(x)==0:
                df = df.append(df2,ignore_index=True)
                continue
            df = df.append(df2[~df2['L3 Cluster ID'].isin(x)],ignore_index=True)
            df2 = df2[df2['L3 Cluster ID'].isin(x)]
            df2['L3 Phrase Count'] = len(df2)
            df2['L3 Review Count'] = df2['Review ID'].nunique()
            df2['L3 Rating (Phrase)'] = df2['Review Rating'].mean()
            df2['L3 Rating (Review)'] = df2.drop_duplicates(['Review ID'])['Review Rating'].mean()    
            df2['L3 Cluster'] = 'Miscellaneous'
            df2['L3 Cluster ID'] = l2+'_misc'#_'+senti[0]
            if len(df2)==1:
                for j in df2.index:
                    df2['L3 Cluster Phrase'][j] = df2['L3 Cluster Phrase'].iloc[0].strip().capitalize()                   
                df = df.append(df2,ignore_index=True)
                continue
            else:
                df2['sim']=''
                for i in df2.index:
                    sim=[]
                    for j in df2.index:
                        if i!=j:
                            sim.append(jellyfish.levenshtein_distance(df2['L4 Phrase'][i],df2['L4 Phrase'][j]))
                    df2['sim'][i] = sum(sim)/len(sim)
                df2['L3 Cluster Phrase'] = df2[(df2['sim']==df2['sim'].min())]['L4 Phrase'].iloc[0].strip().capitalize()                   
            df = df.append(df2,ignore_index=True)
    return df.drop(['sim'],1)

In [12]:
daf.loc[:,['L3 Cluster','Review ID']].drop_duplicates().groupby(['L3 Cluster']).count().sort_values(['Review ID'],ascending=False).head(30).reset_index()

,L3 Cluster,Review ID
0,Skin,15042
1,Buy,9900
2,Foundation,7163
3,Price,3103
4,Cakey,2637
5,Coverage full,2111
6,Last day,1880
7,Easy blend,1679
8,Great coverage,1679
9,Coverage,1594


In [96]:
pd.merge(df.loc[:,['L3 Cluster ID','L3 Cluster']].drop_duplicates(),x,on='L3 Cluster ID').sort_values(['Total_Reviews'],ascending=0).to_excel('quarterly_analysis_20230303v1.xlsx',index=False)

In [92]:
for i in cols['qy'].values:
    x[i+'_ranking'] = x[i+'_#reviews'].rank(ascending=0,method='dense')

In [2]:
df = pd.DataFrame()
for i in range(5,16):
    daf = pd.read_csv('PAIN'+str(i)+' Amazon Reviews.csv')
    df = pd.concat([df,daf],ignore_index=True)
df['Product'] = df['ReviewID'].str.split('R',expand=True)[0]

In [30]:
df['L2 Cluster'].replace({"Skin":"Skin Type",'Purchase':"Repurchase",
                        "Design":"Product Design"},inplace=True)

In [12]:
df.drop(['Sentiment'],1,inplace=True)

In [16]:
d1 = df
d2 = df.iloc[:,:19]
d2['Display Flag'] = d1['Display Flag']
d3 = df.iloc[:,:12]
d4 = df.iloc[:,:6]

# d4['Product'] = d3['Product'] = d2['Product'] = d1['Product']
d2.drop_duplicates(['L2 Cluster','L3 Cluster'],inplace=True)
d3.drop_duplicates(['L2 Cluster'],inplace=True)
d4.drop_duplicates(['L1 Cluster'],inplace=True)

for x in [d1,d2]:
    x['Flag']=''
    for i in x.index:
        if x['L3 Rating (Review)'][i]>=4:
            x['Flag'][i] = '4+'
        else:
            x['Flag'][i] = '4-'
        
#Creating output excel file
name = 'hair_oil_category_detailed_20230323v1.xlsx'
path = r"{fname}".format(fname=name)
with pd.ExcelWriter(path) as engine:
    d1.to_excel(excel_writer=engine,sheet_name='L4',index=False)
    d2.to_excel(excel_writer=engine,sheet_name='L3',index=False)
    d3.to_excel(excel_writer=engine,sheet_name='L2',index=False)
    d4.to_excel(excel_writer=engine,sheet_name='L1',index=False)
print("Excel File with 4 sheets is created.\n")
print("File name:",name)

Excel File with 4 sheets is created.

File name: hair_oil_category_detailed_20230323v1.xlsx


# Competitor Extraction

In [2]:
df = pd.read_excel('05_all_reviews.xlsx')

In [3]:
nlp = spacy.load("en_core_web_trf")
df.fillna('',inplace=True)
df['Text'] = df['ReviewTitle']+' '+df['ReviewBody']
df['Product'] = df['ReviewID'].str.split('R',expand=True)[0]

In [4]:
daf = pd.DataFrame(columns=['Product','ReviewID','Review','Text','Label','Rating'])

for i in range(len(df)):
    doc = nlp(df['Text'][i])
    for entity in doc.ents:
        if (entity.label_=='ORG') or (entity.label_=='PRODUCT'):
            daf.loc[len(daf)] = [df['Product'][i],df['ReviewID'][i],df['Text'][i],entity.text,entity.label_,df['Rating'][i]]
            print(i,entity.label_, ' | ', entity.text)
daf['Text'] = daf['Text'].apply(lambda x:x.lower())

3 ORG  |  kapiva
19 PRODUCT  |  Meal Replacement Shake
36 PRODUCT  |  Kapiva Slim Shake
37 ORG  |  Kapiva
40 PRODUCT  |  Kapiva cofee
44 PRODUCT  |  Milk
47 ORG  |  Amazon
60 PRODUCT  |  Kapiva Slim Shake
83 PRODUCT  |  Kapiva Slim Shake
84 ORG  |  Kapiva
84 ORG  |  Kapiva
88 PRODUCT  |  Kapiva Slim Shake-
88 PRODUCT  |  Kapiva Slim Shake
100 ORG  |  Kapiva
100 ORG  |  kapiva
100 ORG  |  Kapiva
116 PRODUCT  |  Kapiva Slim
123 ORG  |  Kapiva
123 ORG  |  Kapiva
128 ORG  |  Kapiva
133 ORG  |  Nestle
133 PRODUCT  |  Nestle Optifast
156 ORG  |  Kapiva
163 PRODUCT  |  Kapiva
169 ORG  |  Kapiva
169 PRODUCT  |  Meal Replacement Slim Shake
179 PRODUCT  |  Slimshake
180 PRODUCT  |  The Slim Shake
184 PRODUCT  |  Kapiva Slim
184 ORG  |  Kapiva
188 PRODUCT  |  kapiva
189 PRODUCT  |  Shake
192 ORG  |  kapiva
197 ORG  |  Yakk
202 PRODUCT  |  KAPIVA SLIM SHAKE
202 ORG  |  Amazon
204 PRODUCT  |  SlimShake
206 PRODUCT  |  Kapiva Slim Shake
206 PRODUCT  |  Kapiva Slim Shake
207 ORG  |  Kapiva
208 PRODUC

2115 PRODUCT  |  Slimfast
2119 PRODUCT  |  Garcinia Cambogia
2126 PRODUCT  |  NutriNinja
2126 PRODUCT  |  The Chocolate Royale
2130 PRODUCT  |  SlimFast French Vanilla
2146 PRODUCT  |  Slim Fast
2146 PRODUCT  |  Slimfast
2146 PRODUCT  |  Lean Cuisine
2146 PRODUCT  |  Slim Fast
2146 ORG  |  Special K
2146 PRODUCT  |  SlimFast
2146 PRODUCT  |  Mrs. Dash
2146 PRODUCT  |  Skim milk
2147 ORG  |  Walgreens Stores
2151 PRODUCT  |  Slimfast
2158 PRODUCT  |  SlimFast
2162 PRODUCT  |  Slim Fast Meal
2167 PRODUCT  |  Slimfast
2167 PRODUCT  |  Slimfast
2170 ORG  |  Amazon
2172 PRODUCT  |  this Slim Fast
2181 PRODUCT  |  Almond Milk
2182 PRODUCT  |  Slim Fast
2182 PRODUCT  |  Slim Fast
2183 ORG  |  Slimfast
2188 ORG  |  google
2194 PRODUCT  |  Slimfast
2203 PRODUCT  |  Atrafen Thermogenic Fat Burner &
2218 PRODUCT  |  Slim Fast
2218 PRODUCT  |  Slim Fast
2220 PRODUCT  |  Rich Chocolate Royale
2228 PRODUCT  |  Slimfast
2229 ORG  |  Atkins
2230 ORG  |  Amazon
2232 ORG  |  LoseIt
2232 ORG  |  MyFitnes

In [7]:
daf['len'] = daf['Text'].apply(lambda x:len(x))
daf.sort_values(['len'],inplace=True)
daf['Cluster']=''
daf['Score'] = 0
daf.index = range(len(daf))

for i in range(len(daf)-1):
    if daf['Cluster'][i]!='':
        continue
    if daf['Cluster'][i]=='':
        daf['Cluster'][i] = daf['Text'][i]
        daf['Score'][i] = 100
    for j in range(i+1,len(daf)):
        if fuzz.token_set_ratio(daf['Text'][i],daf['Text'][j])>=90:
            daf['Cluster'][j] = daf['Text'][i]
            daf['Score'][j] = fuzz.token_set_ratio(daf['Text'][i],daf['Text'][j])

In [21]:
daf.drop_duplicates(['Cluster','ReviewID']).groupby(['Cluster','Product']).count().reset_index().sort_values(['Cluster','Product','ReviewID'],ascending=False).iloc[:,[0,1,2]].to_excel('slim_shake_product_competitor_summary_20230324v1.xlsx',index=False)

In [13]:
x = daf.groupby(['Cluster']).mean()
x.reset_index(inplace=True)
x['Category Review Count'] = ''
x['Category 4+ Review Count'] = ''
x['Category <4 Review Count'] = ''
x['Category 4+ Review Rating']=''
x['Category <4 Review Rating']=''
for j in daf['Product'].unique():    
    x[j+' 4+ Review Count %'] = ''
    x[j+' <4 Review Count %'] = ''
    
for i in x.index:
    x['Category Review Count'][i] = daf[daf.Cluster==x['Cluster'][i]]['ReviewID'].nunique()
    x['Category 4+ Review Count'][i] = daf[(daf.Cluster==x['Cluster'][i])&(daf['Rating']>=4)]['ReviewID'].nunique()
    x['Category 4+ Review Rating'][i] = daf[(daf.Cluster==x['Cluster'][i])&(daf['Rating']>=4)].drop_duplicates(['ReviewID']).mean()['Rating']
    x['Category <4 Review Count'][i] = x['Category Review Count'][i]-x['Category 4+ Review Count'][i]
    x['Category <4 Review Rating'][i] = daf[(daf.Cluster==x['Cluster'][i])&(daf['Rating']<4)].drop_duplicates(['ReviewID']).mean()['Rating']

for i in x.index:
    for j in daf['Product'].unique():    
        x[j+' 4+ Review Count %'][i] = daf[(daf.Cluster==x['Cluster'][i])&(daf.Product==j)&(daf['Rating']>=4)]['ReviewID'].nunique()
        x[j+' <4 Review Count %'][i] = daf[(daf.Cluster==x['Cluster'][i])&(daf.Product==j)&(daf['Rating']<4)]['ReviewID'].nunique()

for i in x.index:
    for j in daf['Product'].unique():    
        try:
            y = (x[j+' 4+ Review Count %'][i]+x[j+' <4 Review Count %'][i])
            x[j+' 4+ Review Count %'][i] = 100*x[j+' 4+ Review Count %'][i]/y
            x[j+' <4 Review Count %'][i] = 100*x[j+' <4 Review Count %'][i]/y
        except:
            pass
        
x.fillna(0,inplace=True)

for i in daf['Product'].unique():    
    x[i+' 4+ Review 1']=''
    x[i+' 4+ Review 2']=''
    x[i+' <4 Review 1']=''
    x[i+' <4 Review 2']='' 
    
for i in x.index:
    for p in daf['Product'].unique():    
        for j in ['1','2']:
            try:
                x[p+' 4+ Review '+j][i]=daf[(daf['Product']==p)&(daf['Rating']>=4)&(daf.Cluster==x['Cluster'][i])]['Review'].unique()[int(j)-1]
            except:
                pass
            try:
                x[p+' <4 Review '+j][i]=daf[(daf['Product']==p)&(daf['Rating']<4)&(daf.Cluster==x['Cluster'][i])]['Review'].unique()[int(j)-1]
            except:
                pass

In [17]:
x.sort_values(['Category Review Count'],ascending=False).to_excel('slim_shake_competitor_summary_20230323v1.xlsx',index=False)

In [108]:
x['Cluster Review Count'] = x.groupby(['Cluster']).transform("sum")['Category Review Count']

In [66]:
pd.set_option('display.max_rows', None)

In [103]:
query = 'el'
choices = list(x.Text)
process.extract(query, choices,limit=None,scorer=fuzz.token_set_ratio)

[('el', 100),
 ("el's", 100),
 ('el dw ', 100),
 ('el double wear', 100),
 ('el doublewear really does not budge.', 100),
 ('elf', 80),
 ('e.l.', 80),
 ('opel', 67),
 ('belk', 67),
 ('eldw', 67),
 ('veil', 67),
 ('elexa', 57),
 ('pearl', 57),
 ('shell', 57),
 ('de', 50),
 ('weeell', 50),
 ('chanel', 50),
 ('pebble', 50),
 ('stella', 50),
 ('loreal', 50),
 ('rimmel', 50),
 ('revlon', 50),
 ('l’oreal', 44),
 ('elta md', 44),
 ('gerlain', 44),
 ('arsenal', 44),
 ("l'oreal", 44),
 ('la base', 44),
 ('estee l.', 44),
 ('sol', 40),
 ('med', 40),
 ('l43', 40),
 ('one', 40),
 ('ole', 40),
 ('dlw', 40),
 ('tlc', 40),
 ('ysl', 40),
 ('def', 40),
 ('elephant', 40),
 ('guerlain', 40),
 ('poreless', 40),
 ('dermacol', 40),
 ('wetnwild', 40),
 ('idealist', 40),
 ('shell 1c0', 36),
 ('porcelain', 36),
 ('temptalia', 36),
 ('exxon oil', 36),
 ('covergirl', 36),
 ('1c0 shell', 36),
 ('cool bone', 36),
 ('4w4 hazel', 36),
 ('real tech.', 36),
 ('oily', 33),
 ('ecru', 33),
 ('demi', 33),
 ('bone', 33),
 

# Monthly Review Count

In [204]:
dr = pd.date_range('2012-01-01','2023-02-01',freq='M')
col = pd.Series(dr.month).astype(str)+'-'+pd.Series(dr.year).astype(str)
daf = pd.DataFrame(columns=['client']+list(col))
d = {'P1':'P1_Estee Lauder.csv','P2':'P2.csv','P4':'P4 First Aid Beauty.csv','P5':'P5.csv','P6':'P6.csv'}
daf['client'] = d.keys()

for i in range(len(daf['client'])):
    df = pd.read_csv(d[daf['client'][i]])
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values(['Date'],inplace=True)
    df['year'] = df['Date'].apply(lambda x:x.year)
    df['month'] = df['Date'].apply(lambda x:x.month)
    for j in col:
        month = int(j.split('-')[0])
        year = int(j.split('-')[1])
        daf[j][i] = len(df[(df['year']==year) & (df['month']==month)])
daf.to_excel('review_count_20230130v2.xlsx',index=False)

In [4]:
pd.merge(df,daf.iloc[:,1:],on='Review ID').to_excel('P4_detailed_20230124v2.xlsx',index=False)

# L3 Clusters Cross Connection

In [ ]:
df = pd.read_excel('P4_detailed_20230201v1.xlsx')

In [ ]:
for senti in ['Positive','Negative']:
    print(senti)
    x = df[df['Sentiment']==senti]
    connection = [[-10]*x['L3 Cluster ID'].nunique()]*x['L3 Cluster ID'].nunique()
    connection = pd.DataFrame(connection,columns=x['L3 Cluster ID'].unique(),index=x['L3 Cluster ID'].unique())

    for i in x['L3 Cluster ID'].unique():
        for j in x['L3 Cluster ID'].unique():
            if i!=j and connection[i][j]==-10:
                lst1 = x[x['L3 Cluster ID']==i]['Review ID'].unique()
                lst2 = x[x['L3 Cluster ID']==j]['Review ID'].unique()
                connection[i][j] = len(set(lst1) & set(lst2))*100/min(len(lst1),len(lst2))
                connection[j][i] = connection[i][j]

    col = []
    for i in connection.columns:
        col.append((x[x['L3 Cluster ID']==i]['L2 Cluster']+'-'+x[x['L3 Cluster ID']==i]['L3 Cluster']).unique()[0])
    connection.columns = col
    connection.index = col
    
    sns.set_style('darkgrid')
    fig, ax = plt.subplots(figsize=(32, 24))
    fig = sns.heatmap(connection,annot=True,cmap = 'rainbow', linewidth = 0.1 ,ax=ax)
    results_path = 'P4_'+senti+'.png'
    fig.get_figure().savefig(results_path)

# Mapping in DataFrame

In [18]:
tags = pd.read_csv('Review Categories - For C2.csv')
tags.drop(['Unnamed: 0','Unnamed: 3'],1,inplace=True)
tags.drop(0,inplace=True)
tags['Not-Preferred Tagging'] = tags['Not-Preferred Tagging'].str.lower()
tags['Not-Preferred Tagging'] = tags['Not-Preferred Tagging'].str.strip()
tags['Preferred Tagging'] = tags['Preferred Tagging'].str.lower()
tags['Preferred Tagging'] = tags['Preferred Tagging'].str.strip()

tags.loc[len(tags.index)+1] = ['skin','skintype']
tags.loc[len(tags.index)+1] = ['skin','skinissue']
tags.loc[len(tags.index)+1] = ['odor','scent']
tags.loc[len(tags.index)+1] = ['application','usage']
tags.loc[len(tags.index)+1] = ['application','change']
tags.loc[len(tags.index)+1] = ['packaging','shipping'] 

for i in range(len(df)):
    try:
        df['Aspect'][i] = list(tags[tags['Not-Preferred Tagging']==df['Aspect'][i]]['Preferred Tagging'])[0]
    except:
        df['Aspect'][i] = df['Aspect'][i]

for i in df.index:
    if df['Aspect'][i]=='shipping':
        df['Aspect'][i]='packaging'

df['Aspect'] = df['Aspect'].apply(lambda x:x.capitalize())

In [67]:
txt_to_jsonl("sample.jsonl")

# Aspect count per sentiment

In [36]:
n_aspect={}
p_aspect={}

for i in df2.index:
    for senti in ['Positive','Negative']:
        l = df2[senti][i]
    #     print(i)
        if l is None or (len(l)==0 or len(l)==1):
            continue
        if l[0]=='':
            l.pop(0)
        if senti=='Negative':
            for j in l:
                try:
                    n_aspect[j.split(':')[0]]+=1           
                except:
                    n_aspect[j.split(':')[0]]=1           
        else:
            for j in l:
                try:
                    p_aspect[j.split(':')[0]]+=1           
                except:
                    p_aspect[j.split(':')[0]]=1 
del p_aspect['None']
del n_aspect[' END']

negative_aspects = pd.DataFrame(n_aspect.values(),index=n_aspect.keys(),columns=['count'])
positive_aspects = pd.DataFrame(p_aspect.values(),index=p_aspect.keys(),columns=['count'])

negative_aspects.to_csv('negative_aspects_count.csv')
positive_aspects.to_csv('positive_aspects_count.csv')

KeyError: 'Positive'

# completion df -> txt file

In [ ]:
df2.fillna('',inplace=True)
df2['prompt']='##'+df2['Review ID']+'\n'+df2['Review Title']+'\n'+df2['Review']+'##ENDING##\n\n\n'+df2['completion']+'\n'
st = ''
for i in df2.index:
    st=st+df2['prompt'][i]
# print(st)

text_file = open("output_txt.txt", "w",encoding="utf8") 
text_file.write(st)
text_file.close()

# df -> Docano Output

In [150]:
def getIndex(text, phrase):
    text, phrase = text.lower(), phrase.lower()
    text, phrase = text.strip(), phrase.strip()
    
    if text.find(phrase) != -1:
        return text.find(phrase)
    
    lastWord = phrase.split()[-1]
    lastWordIndex = text.find(lastWord)
    
    if lastWordIndex != -1:
        return lastWordIndex - (len(phrase) - len(lastWord))
    
    firstWord = phrase.split()[0]
    firstWordIndex = text.find(firstWord)
    
    if firstWordIndex != -1:
        return firstWordIndex
    
    return -1

df3["startIndex"] = df3.apply(lambda x : getIndex(x["Review"], x["phrase"]), axis = 1)
df3 = df3[df3.startIndex != -1]
df3['endIndex'] = df3.apply(lambda x : x['startIndex']+len(x['phrase']), axis = 1)
df3['pl'] = df3.apply(lambda x:len(x["phrase"].split(' '))-1,1)
df3.drop(df3[df3.pl==0].index,inplace=True)

df3['sentiment'] = df3['sentiment'].str.replace('Positive','pos')
df3['sentiment'] = df3['sentiment'].str.replace('Negative','neg')
        
df3['id'] = df3['Review ID'].str.split('P4R',expand=True).iloc[:,[1]]
df3['Tag'] = df3['sentiment']+'_'+df3['aspect']
df3.rename(columns={'Review':'text'},inplace=True)
df3['id'] = df3['id'].astype(int)
df3.sort_values(['id','startIndex'],inplace=True)

df3['entity'] = ''
for i in df3.index:
    l = []
    l.append(int(df3['startIndex'][i]))
    l.append(int(df3['endIndex'][i]))
    l.append(df3['Tag'][i])
    df3['entity'][i] = l
    
# df3.drop(['Review ID','sentiment','aspect','Tag','startIndex','endIndex','Phrase'],1,inplace=True)

keyList = ["id", "text", "Date","Rating","cats","entities","Comments"]

dl = []
for id in df3['id'].unique():
    d = {}
    for i in keyList:
        d[i] = None
    d['id'] = int(id)
    d['text'] = list(df3[df3['id']==id]['text'])[0]
    d['Date'] = list(df3[df3['id']==id]['Date'])[0]
    d['Rating'] = list(df3[df3['id']==id]['Rating'])[0]
    d['cats'] = []
    d['entities'] = list(df3[df3['id']==id]['entity'])
    d['Comments'] = []
    dl.append(d)

# with open("doccano_op_20221226v3.jsonl", 'w', encoding="utf8") as f:
#     for item in dl:
#         f.write(json.dumps(item) + "\n")

In [27]:
df3 = df3[(~df3['Review ID'].isin(l1)) & (~df3.aspect.isin(['Temperature','Super dry skin','Substitute','Sent','Motivator','Age', 'Customer', ' END']))]

In [9]:
l1 = list(df['Review ID'].unique())
l2 = list(df3['Review ID'].unique())

In [30]:
len(set(l1+l2))

1312

In [ ]:
df3['label']=''
d = {}
l = list(df3.aspect.unique())
for i in range(len(l)):
    d[l[i]]=i
for i in df3.index:
    df3['label'][i] = d[df3['aspect'][i]]

# Adding a week column in reviews file

In [ ]:
df = pd.read_csv('P1_Estee Lauder.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['week_of']=''

for i in df.index:
    x = df['Date'][i].weekday()
    if x==6:
        df['week_of'][i] = df['Date'][i]#.strftime('%d%b%Y')
    else:
        df['week_of'][i] = (df['Date'][i]-timedelta(x+1))#.strftime('%d%b%Y')
        
# df.to_csv('reviews_w_week_20221227v2.csv',index=False)

# Weekly Summary

In [ ]:
daf = df.join(pd.get_dummies(df['Rating'])).groupby('week_of').sum().iloc[:,2:]
daf = daf.join(df.groupby('week_of').count().loc[:,['Review ID']])
daf.rename(columns = {'Review ID':'#_reviews'}, inplace = True)
daf = daf.iloc[:,[5,0,1,2,3,4]]

l = list(daf.columns)[1:]
for i in l:
    name = '% of '+i
    daf[name] = round(100*(daf[i]/daf['#_reviews']),1)
daf.sort_values(['week_of'],ascending=False,inplace=True)
daf = daf.reset_index()
for i in daf.index:
    daf['week_of'][i] = daf['week_of'][i].strftime('%d_%b_%Y')
# daf.to_csv('week_summary_20221227v1.csv',index=False)